Lambda School Data Science

*Unit 4, Sprint 1, Module 4*

---

# Topic Modeling (Prepare)

On Monday we talked about summarizing your documents using just token counts. Today, we're going to learn about a much more sophisticated approach - learning 'topics' from documents. Topics are a latent structure. They are not directly observable in the data, but we know they're there by reading them.

> **latent**: existing but not yet developed or manifest; hidden or concealed.

## Use Cases
Primary use case: what the hell are your documents about? Who might want to know that in industry - 
* Identifying common themes in customer reviews
* Discovering the needle in a haystack 
* Monitoring communications (Email - State Department) 

## Learning Objectives
*At the end of the lesson you should be able to:*
* <a href="#p1">Part 1</a>: Describe how an LDA Model works
* <a href="#p2">Part 2</a>: Estimate a LDA Model with Gensim
* <a href="#p3">Part 3</a>: Interpret LDA results
* <a href="#p4">Part 4</a>: Select the appropriate number of topics


# Latent Dirchilet Allocation Models (Learn)
<a id="#p1"></a>

## Overview
LDA is a "generative probabilistic model". 

Let's play with a model available [here](https://lettier.com/projects/lda-topic-modeling/)

## Follow Along

## Challenge 

# Estimating LDA Models with Gensim (Learn)
<a id="#p1"></a>

## Overview
### A Literary Introduction: *Jane Austen V. Charlotte Bronte*
Despite being born nearly forty years apart, modern fans often pit Jane Austen & Charlotte Bronte against one another in a battle for literary  supremacy. The battle centers around the topics of education for women, courting, and marriage. The authors' similar backgrounds naturally draw comparisons, but the modern fascination is probably due to novelty of British women publishing novels during the early 19th century. 

Can we help close a literary battle for supremacy and simply acknowledge that the authors addressed different topics and deserve to be acknowledged as excellent authors each in their own right?

We're going to apply Latent Dirichlet Allocation a machine learning algorithm for topic modeling to each of the author's novels to compare the distribution of topics in their novels.

In [1]:
import numpy as np
import gensim
import os
import re

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore

import pandas as pd

In [2]:
gensim.__version__

'3.8.1'

### Novel Data
I grabbed the novel data pre-split into a bunch of smaller chunks

In [3]:
path = './data/austen-brontë-split'

### Text Preprocessing
**Activity**: update the function `tokenize` with any technique you have learned so far this week. 

In [4]:
# 1) Plain Python - ''.split command
# 2) Spacy - just the lemmas from the document
# 3) Gensim - simple_preprocess

STOPWORDS = set(STOPWORDS).union(set(['said', 'mr', 'mrs']))

def tokenize(text):
    return [token for token in simple_preprocess(text) if token not in STOPWORDS]

In [5]:
import os

def gather_data(path_to_data): 
    data = []
    for f in os.listdir(path):
        if os.path.isdir(f) == False:
            if f[-3:] == 'txt':
                with open(os.path.join(path,f)) as t:
                    text = t.read().strip('\n')
                    data.append(tokenize(str(text)))       
    return data

In [6]:
tokens = gather_data(path)

In [7]:
tokens[0][0:10]

['attentions',
 'spring',
 'score',
 'let',
 'sit',
 'understand',
 'miss',
 'bennet',
 'came',
 'determined']

In [8]:
"this is a sample string with a \n newline character".replace('\n', '')

'this is a sample string with a  newline character'

## Follow Along

In [9]:
titles = [t[:-4] for t in os.listdir(path) if os.path.isdir(t) == False]

In [10]:
len(titles)

813

In [11]:
len(tokens)

813

### Author DataFrame


In [12]:
df = pd.DataFrame(index=titles, data={'tokens':tokens})

In [13]:
df.head()

,tokens
Austen_Pride0109,"[attentions, spring, score, let, sit, understa..."
CBronte_Villette0114,"[merely, getting, better, view, face, struck, ..."
Austen_Sense0038,"[patience, melancholy, state, person, expectat..."
Austen_Emma0133,"[ready, believe, character, improve, acquire, ..."
CBronte_Jane0002,"[dim, bleared, eye, flabby, cheeks, ought, sch..."


In [14]:
df['author'] = df.reset_index()['index'].apply(lambda x: x.split('_')[0]).tolist()
df['book'] = df.reset_index()['index'].apply(lambda x: x.split('_')[1][:-4]).tolist()
df['section'] = df.reset_index()['index'].apply(lambda x: x[-4:]).tolist()
df['section'] = df['section'].astype('int')

In [15]:
df['author'] = df['author'].map({'Austen':1, 'CBronte':0})

In [16]:
df.author.value_counts()

0    441
1    372
Name: author, dtype: int64

In [17]:
df.head()

,tokens,author,book,section
Austen_Pride0109,"[attentions, spring, score, let, sit, understa...",1,Pride,109
CBronte_Villette0114,"[merely, getting, better, view, face, struck, ...",0,Villette,114
Austen_Sense0038,"[patience, melancholy, state, person, expectat...",1,Sense,38
Austen_Emma0133,"[ready, believe, character, improve, acquire, ...",1,Emma,133
CBronte_Jane0002,"[dim, bleared, eye, flabby, cheeks, ought, sch...",0,Jane,2


### Streaming Documents
Here we use a new pythonic thingy: the `yield` statement in our function. This allows us to iterate over a bunch of documents without actually reading them into memory. You can see how we use this function later on. 

In [18]:
def doc_stream(path):
    for f in os.listdir(path):
        if os.path.isdir(f) == False:
            if f[-3:] == 'txt':
                with open(os.path.join(path,f)) as t:
                    text = t.read().strip('\n')
                    tokens = tokenize(text)
                yield tokens

In [19]:
streaming_data = doc_stream(path)

In [20]:
type(streaming_data)

generator

In [21]:
# gather_data => returns a list
# doc_stream => returns a generator

In [ ]:
next(streaming_data) # Returns one document at a time from the generator

### Gensim LDA Topic Modeling

In [23]:
# A Dictionary Representation of all the words in our corpus
id2word = corpora.Dictionary(doc_stream(path))

In [24]:
id2word.token2id['england']

1368

In [26]:
id2word.doc2bow(tokenize("This is a sample message Darcy England England England"))

[(49, 1), (476, 1), (1368, 3), (13639, 1)]

In [27]:
import sys
print(sys.getsizeof(id2word))
print(sys.getsizeof(tokens))

56
7056


In [28]:
len(id2word.keys())

22095

In [29]:
# Let's remove extreme values from the dataset
id2word.filter_extremes(no_below=5, no_above=0.95)

In [30]:
len(id2word.keys())

8102

In [31]:
# a bag of words(bow) representation of our corpus
# Note: we haven't actually read any text into memory here
# Although abstracted away - tokenization IS happening in the doc_stream f(x)
corpus = [id2word.doc2bow(text) for text in doc_stream(path)]

In [33]:
corpus[345][:10]

[(8, 1),
 (18, 1),
 (34, 1),
 (40, 1),
 (55, 1),
 (59, 2),
 (61, 1),
 (68, 1),
 (71, 1),
 (74, 1)]

In [64]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   random_state=723812,
                   num_topics = 15,
                   passes=10,
                   workers=12
                  )

In [65]:
lda.print_topics()

[(0,
  '0.010*"harriet" + 0.010*"miss" + 0.007*"elton" + 0.006*"emma" + 0.005*"scatcherd" + 0.005*"burns" + 0.005*"good" + 0.005*"thought" + 0.004*"read" + 0.004*"day"'),
 (1,
  '0.022*"elinor" + 0.018*"marianne" + 0.009*"sister" + 0.009*"edward" + 0.008*"dashwood" + 0.008*"mother" + 0.007*"jennings" + 0.007*"willoughby" + 0.006*"time" + 0.006*"lucy"'),
 (2,
  '0.025*"harriet" + 0.019*"elton" + 0.017*"miss" + 0.016*"emma" + 0.011*"jane" + 0.009*"woodhouse" + 0.008*"think" + 0.007*"dear" + 0.007*"oh" + 0.007*"thing"'),
 (3,
  '0.023*"sir" + 0.020*"jane" + 0.016*"rochester" + 0.008*"shall" + 0.006*"love" + 0.006*"mason" + 0.006*"yes" + 0.005*"like" + 0.004*"look" + 0.004*"little"'),
 (4,
  '0.009*"john" + 0.007*"little" + 0.006*"know" + 0.005*"like" + 0.005*"good" + 0.005*"st" + 0.005*"think" + 0.004*"tell" + 0.004*"dr" + 0.004*"come"'),
 (5,
  '0.009*"jane" + 0.009*"elizabeth" + 0.007*"miss" + 0.007*"bingley" + 0.007*"sister" + 0.006*"soon" + 0.006*"darcy" + 0.005*"lady" + 0.004*"happy"

In [66]:
words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]

In [67]:
topics = [' '.join(t[0:5]) for t in words]

In [68]:
topics

['harriet miss elton emma scatcherd',
 'elinor marianne sister edward dashwood',
 'harriet elton miss emma jane',
 'sir jane rochester shall love',
 'john little know like good',
 'jane elizabeth miss bingley sister',
 'emma miss thing know think',
 'jane know man good miss',
 'elizabeth darcy bennet bingley jane',
 'little house time day room',
 'bessie reed john georgiana like',
 'like hunsden good little thought',
 'like night rochester door come',
 'little like madame thought hand',
 'graham polly bretton little sir']

In [73]:
lda.print_topics()[0]

(0,
 '0.010*"harriet" + 0.010*"miss" + 0.007*"elton" + 0.006*"emma" + 0.005*"scatcherd" + 0.005*"burns" + 0.005*"good" + 0.005*"thought" + 0.004*"read" + 0.004*"day"')

In [38]:
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
harriet miss elton emma scatcherd

------ Topic 1 ------
elinor marianne sister edward dashwood

------ Topic 2 ------
harriet elton miss emma jane

------ Topic 3 ------
sir jane rochester shall love

------ Topic 4 ------
john little know like good

------ Topic 5 ------
jane elizabeth miss bingley sister

------ Topic 6 ------
emma miss thing know think

------ Topic 7 ------
jane know man good miss

------ Topic 8 ------
elizabeth darcy bennet bingley jane

------ Topic 9 ------
little house time day room

------ Topic 10 ------
bessie reed john georgiana like

------ Topic 11 ------
like hunsden good little thought

------ Topic 12 ------
like night rochester door come

------ Topic 13 ------
little like madame thought hand

------ Topic 14 ------
graham polly bretton little sir



## Challenge 

You will apply an LDA model to a customer review dataset to practice the fitting and estimation of LDA. 

# Interpret LDA Results (Learn)
<a id="#p3"></a>

## Overview

## Follow Along

### Topic Distance Visualization

In [39]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

In [40]:
pyLDAvis.gensim.prepare(lda, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
13    -0.076950  0.016633       1        1  18.292974
6      0.108964  0.004955       2        1  17.964882
8      0.119813 -0.040144       3        1  12.176672
12    -0.113992 -0.029894       4        1  10.529463
1      0.118481 -0.070619       5        1  10.005984
11    -0.051676 -0.014619       6        1   8.847969
9     -0.107007 -0.002538       7        1   5.772343
4     -0.042355 -0.025895       8        1   4.922864
2      0.123666  0.032073       9        1   3.437604
3     -0.078403 -0.080810      10        1   3.277903
10    -0.042692 -0.040807      11        1   2.121842
7     -0.003258 -0.007387      12        1   1.236979
5      0.093039  0.012019      13        1   0.559786
14    -0.068885  0.111515      14        1   0.478093
0      0.021255  0.135518      15        1   0.374635, topic_info=           Term         Freq        Total Category  logprob  loglift
867        emma   836.000000   836.000000  Default  30.0000  30.0000
142        miss  1508.000000  1508.000000  Default  29.0000  29.0000
65    elizabeth   643.000000   643.000000  Default  28.0000  28.0000
641      elinor   703.000000   703.000000  Default  27.0000  27.0000
910        jane   914.000000   914.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
700      little     2.996991  1714.961426  Topic15  -5.9708  -0.7626
1376       come     2.693250   829.710632  Topic15  -6.0776  -0.1434
896        home     2.461176   490.950165  Topic15  -6.1677   0.2913
223       think     2.553533  1305.546753  Topic15  -6.1309  -0.6499
247       young     2.457127   661.967712  Topic15  -6.1694  -0.0093

[1117 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
4403      6  0.177513  abandon
4403      8  0.710052  abandon
801       2  0.814382    abbey
801       9  0.130301    abbey
801      15  0.032575    abbey
...     ...       ...      ...
247      14  0.003021    young
247      15  0.003021    young
5676      1  0.962493    zelie
2045      1  0.284161  zoraide
2045      6  0.674882  zoraide

[4232 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[14, 7, 9, 13, 2, 12, 10, 5, 3, 4, 11, 8, 6, 15, 1])

### Overall Model / Documents

In [61]:
lda[corpus[0]]

[(8, 0.99730086)]

In [42]:
distro = [lda[d] for d in corpus]

In [43]:
distro[0]

[(8, 0.99730086)]

In [44]:
distro = [lda[d] for d in corpus]

def update(doc):
        d_dist = {k:0 for k in range(0,15)}
        for t in doc:
            d_dist[t[0]] = t[1]
        return d_dist
    
new_distro = [update(d) for d in distro]

In [ ]:
new_distro

In [45]:
len(new_distro)

813

In [46]:
df.head()

,tokens,author,book,section
Austen_Pride0109,"[attentions, spring, score, let, sit, understa...",1,Pride,109
CBronte_Villette0114,"[merely, getting, better, view, face, struck, ...",0,Villette,114
Austen_Sense0038,"[patience, melancholy, state, person, expectat...",1,Sense,38
Austen_Emma0133,"[ready, believe, character, improve, acquire, ...",1,Emma,133
CBronte_Jane0002,"[dim, bleared, eye, flabby, cheeks, ought, sch...",0,Jane,2


In [47]:
df = pd.DataFrame.from_records(new_distro, index=titles)
df.columns = topics
df['author'] = df.reset_index()['index'].apply(lambda x: x.split('_')[0]).tolist()

In [48]:
df.head()

,harriet miss elton emma scatcherd,elinor marianne sister edward dashwood,harriet elton miss emma jane,sir jane rochester shall love,john little know like good,jane elizabeth miss bingley sister,emma miss thing know think,jane know man good miss,elizabeth darcy bennet bingley jane,little house time day room,bessie reed john georgiana like,like hunsden good little thought,like night rochester door come,little like madame thought hand,graham polly bretton little sir,author
Austen_Pride0109,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.997301,0.0,0.000000,0.0,0.000000,0.000000,0.0,Austen
CBronte_Villette0114,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.997737,0.0,CBronte
Austen_Sense0038,0.0,0.99758,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,Austen
Austen_Emma0133,0.0,0.00000,0.0,0.0,0.0,0.0,0.997516,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,Austen
CBronte_Jane0002,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.571791,0.0,0.425865,0.000000,0.0,CBronte


In [49]:
df.groupby('author').mean()

,harriet miss elton emma scatcherd,elinor marianne sister edward dashwood,harriet elton miss emma jane,sir jane rochester shall love,john little know like good,jane elizabeth miss bingley sister,emma miss thing know think,jane know man good miss,elizabeth darcy bennet bingley jane,little house time day room,bessie reed john georgiana like,like hunsden good little thought,like night rochester door come,little like madame thought hand,graham polly bretton little sir
author,,,,,,,,,,,,,,,
Austen,0.005016,0.216153,0.074920,0.000424,0.004121,0.011434,0.376432,0.006290,0.282188,0.009106,0.005224,0.002296,0.002654,0.001141,0.000000
CBronte,0.002447,0.001725,0.000278,0.064434,0.088338,0.000891,0.018715,0.018737,0.004895,0.092857,0.035181,0.156845,0.188935,0.314925,0.008162


## Challenge
### *Can we see if one of the authors focus more on men than women?*

*  Use Spacy for text preprocessing
*  Extract the Named Entities from the documents using Spacy (command is fairly straight forward)
*  Create unique list of names from the authors (you'll find that there are different types of named entities not all people)
*  Label the names with genders (can you this by hand or you use the US census name lists)
*  Customize your processing to replace the proper name with your gender from the previous step's lookup table
*  Then follow the rest of the LDA flow


# Selecting the Number of Topics (Learn)
<a id="#p4"></a>

## Overview

## Follow Along

In [54]:
from gensim.models.coherencemodel import CoherenceModel

def compute_coherence_values(dictionary, corpus, limit, start=2, step=3, passes=5):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    limit : Max num of topics
    passes: the number of times the entire lda model & coherence values are calculated

    Returns:
    -------
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    
    coherence_values = []
    
    for iter_ in range(passes):
        for num_topics in range(start, limit, step):
            model = LdaMulticore(corpus=corpus, num_topics=num_topics, id2word=dictionary, workers=4)
            coherencemodel = CoherenceModel(model=model,dictionary=dictionary,corpus=corpus, coherence='u_mass')
            coherence_values.append({'pass': iter_, 
                                     'num_topics': num_topics, 
                                     'coherence_score': coherencemodel.get_coherence()
                                    })

    return coherence_values

In [ ]:
# Can take a long time to run.
coherence_values = compute_coherence_values(dictionary=id2word, 
                                                        corpus=corpus,
                                                        start=2, 
                                                        limit=40, 
                                                        step=2,
                                                        passes=10)

In [89]:
topic_coherence = pd.DataFrame.from_records(coherence_values)

In [ ]:
topic_coherence.head()

In [ ]:
import seaborn as sns

ax = sns.lineplot(x="num_topics", y="coherence_score", data=topic_coherence)

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [ ]:
lda[id2word.doc2bow(tokenize("This is a sample document to score with a topic distribution."))]

# Sources

### *References*
* [Andrew Ng et al paper on LDA](https://ai.stanford.edu/~ang/papers/jair03-lda.pdf)
* On [Coherence](https://pdfs.semanticscholar.org/1521/8d9c029cbb903ae7c729b2c644c24994c201.pdf)

### *Resources*

* [Gensim](https://radimrehurek.com/gensim/): Python package for topic modeling, nlp, word vectorization, and few other things. Well maintained and well documented.
* [Topic Modeling with Gensim](http://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#11createthedictionaryandcorpusneededfortopicmodeling): A kind of cookbook for LDA with gensim. Excellent overview, but the you need to be aware of missing import statements and assumed prior knowledge.
* [Chinese Restuarant Process](https://en.wikipedia.org/wiki/Chinese_restaurant_process): That really obscure stats thing I mentioned... 
* [PyLDAvis](https://github.com/bmabey/pyLDAvis): Library for visualizing the topic model and performing some exploratory work. Works well. Has a direct parrell implementation in R as well. 
* [Rare Technologies](https://rare-technologies.com/): The people that made & maintain gensim and a few other libraries.
* [Jane Austen v. Charlotte Bronte](https://www.literaryladiesguide.com/literary-musings/jane-austen-charlotte-bronte-different-alike/)